In [1]:
import numpy as np
from typing import Literal, List

In [2]:
sea_level_pressure = 14.7
PSI_TO_DBAR = 0.6894759
DBAR_TO_PSI = 1.450377

In [3]:
class PressureCoefficients:
    C1: float
    C2: float
    C3: float
    D1: float
    D2: float
    T1: float
    T2: float
    T3: float
    T4: float
    T5: float
    Slope: float
    Offset: float
    AD590M: float
    AD590B: float
    
pres_coefs = PressureCoefficients()
pres_coefs.C2 = -1.972709e-001
pres_coefs.C3 = 1.365930e-002

pres_coefs.D1 = 3.604100e-002
pres_coefs.D2 = 0.000000e+000

pres_coefs.T1 = 3.046128e+001
pres_coefs.T2 = -3.791930e-004
pres_coefs.T3 = 4.202260e-006
pres_coefs.T4 = 3.386190e-009
pres_coefs.T5 = 0.000000e+000

pres_coefs.Slope = 1.00000000
pres_coefs.Offset = -0.20000
pres_coefs.AD590M = 1.279410e-002
pres_coefs.AD590B = -9.215740e+000

In [4]:
def pressure_from_frequency(
    pressure_counts: np.ndarray, 
    temp_counts: np.ndarray, 
    coefs: PressureCoefficients,
    units: Literal["dbar", "psia"] = "psia"
):
    """
    Berechnet den Druck [psia] aus Digiquartz-Frequenz und Temperaturcount (arrays erlaubt).

    :param freq_hz: gemessene Druckfrequenz [Hz]
    :param temp_freq_hz: gemessene Temperaturcount
    :param coefs: PressureCoefficients
    :return: Druck in psia
    """

    U = coefs.AD590M * temp_counts + coefs.AD590B

    C = coefs.C1 + coefs.C2 * U + coefs.C3 * U **2
    D = coefs.D1 + coefs.D2 * U
    T0 = (
        coefs.T1 
        + coefs.T2 * U 
        + coefs.T3 * U **2 
        + coefs.T4 * U **3 
        + coefs.T5 * U **4
    )
    # Pressure Periode in µs
    T = 10**6 / pressure_counts
    
    pressure = C * (1 - (T0**2/T**2)) * (1 - D * (1 -(T0**2/T**2)))
    pressure = pressure * coefs.Slope + (coefs.Offset * DBAR_TO_PSI) - sea_level_pressure
    
    if units == "dbar":
        pressure *= PSI_TO_DBAR
    
    return pressure

In [5]:
pressure_count = np.array([32837.55859375, 32837.5859375,  32837.5859375,  32837.55859375, 32837.6171875, 32850.2109375, 32850.1640625, 32850.18359375])  # Druckfreqenz
temperature_count = np.array([1238, 1238, 1238, 1238, 1238, 1234, 1235, 1234]) # Druck count

pressure = pressure_from_frequency(pressure_count, temperature_count, coefs=pres_coefs, units="dbar")
print("Druck in db:",pressure)

Druck in db: [ 1.14901189  1.19740283  1.19740283  1.14901189  1.25270682 23.5767095
 23.4858149  23.52829721]
